In [ ]:
# scRNA-seq Analysis Pipeline

This notebook performs:
- Plotting based on differential analysis

## Requirements
- scanpy
- matplotlib

In [ ]:
adata_all = colon_data.concatenate(
    spleen_data,
    batch_key="tissue",
    batch_categories=["Colon", "Spleen"],
    join="outer"
)

adata_all.obs["condition"] = adata_all.obs["batch"].map({
    "colon_WT": "Wt",
    "colon_KO": "IkBa IEC KO",
    "spleen_WT": "Wt",
    "spleen_KO": "IkBa IEC KO",    
})

In [ ]:
genes = [
    "Cdk8",
    "Fhip2b",
    "Gm7283",
    "Btf3",
    "Ftl1",
    "Gdpd3",
]

genes_wide = [
    "Rgs10", "Atrn1", "AW112010", "Sap20l", "Ltb", "Tle5",
    "H2-T10", "Rpl29", "Cebpb", "Mcl1", "Nfkbia", "Ftl1-ps1",
    "Crebzf", "H2-Q7", "H2-K1", "Ftl1", "Gdpd3"
]

from matplotlib.colors import LinearSegmentedColormap
custom_cmap = LinearSegmentedColormap.from_list(
    "blue_purple_gray",
    ["#d9d9d9", "#c6b7ff", "#1f4aff"]
)

In [ ]:
# Fig.6K

def style_dotplot(ax):
    for label in ax["mainplot_ax"].get_yticklabels():
        label.set_fontstyle("italic")
        if label.get_text() in ["Ftl1", "Gdpd3"]:
            label.set_color("red")

for genelist in [genes, genes_wide]:
    plot = sc.pl.dotplot(
    adata_all,
    var_names=genelist,
    groupby=["tissue", "condition"],
    standard_scale="var",
    color_map=custom_cmap,
    swap_axes=True,
    #log=True,
    dot_max=0.8,
    show=False)

    style_dotplot(plot)
    plt.show()

In [ ]:
# Fig. 6j

adata_all_hm = hm.run_harmony(adata_all.obsm['X_pca'], adata_all.obs, 'tissue')

adata_all.obsm['X_pca_harmony'] = adata_all_hm.Z_corr.T
sc.pp.neighbors(adata_all)
sc.tl.umap(adata_all)
sc.tl.leiden(adata_all)

sc.pl.umap(adata_all, color=['tissue', 'batch'], size=20)

adata_all.obs["samples"] = adata_all.obs["batch"].map({
    "colon_WT": "colon Wt",
    "colon_KO": "colon IkBa IEC KO",
    "spleen_WT": "spleen Wt",
    "spleen_KO": "spleen IkBa IEC KO",    
})

# Copy adata to avoid modifying original
adata_plot = adata_all.copy()

# Create a new categorical column for plotting
adata_plot.obs['highlight'] = 'Other'  # default for all cells

# Mask for Activated CD4+ T cells
mask = adata_plot.obs['majority_voting'] == 'Activated CD4+ T cell' #choose whatever you want

# Assign their actual sample labels
adata_plot.obs.loc[mask, 'highlight'] = adata_plot.obs.loc[mask, 'samples']

# Define the palette
palette = {
    'colon Wt': 'magenta',
    'colon IkBa IEC KO': 'blue',
    'spleen Wt': 'green',
    'spleen IkBa IEC KO': 'black',
    'Other': 'lightgray'
}

# Plot UMAP
sc.pl.umap(
    adata_plot,
    color='highlight',
    palette=palette,
    size=50,
    frameon=False,
    legend_loc='right margin',
    title='CD4+ T cells'
)

In [ ]:
# Fig. 6I

sc.pl.violin(
    adata_all,
    keys='Gdpd3',                
    groupby='samples', 
    rotation=90, 
    palette={
        'colon Wt': 'lightgrey',
        'colon IkBa IEC KO': 'coral',
        'spleen Wt': 'lightgrey',
        'spleen IkBa IEC KO': 'turquoise'
    },
    stripplot=True,             
    size=0.3                    
)